In [1]:
import os
from pathlib import Path

In [2]:
%pwd

'd:\\CDAC\\Machine Learning\\sentiment-analyzer\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\CDAC\\Machine Learning\\sentiment-analyzer'

In [22]:
from dataclasses import dataclass

In [23]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_dir: Path
    unzip_dir: Path
    dataset_file: Path
    train_data_dir: Path
    test_data_dir: Path
    encoding: str
    train_size: int
    test_size: int

In [24]:
from sentimentAnalyzer.constant import *
from sentimentAnalyzer.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        dataset = self.params.dataset
        create_directories([config.root_dir])
        create_directories([config.train_data_dir])
        create_directories([config.test_data_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_dir = config.source_dir,
            unzip_dir = config.unzip_dir,
            dataset_file = config.dataset_file,
            train_data_dir = config.train_data_dir,
            test_data_dir = config.test_data_dir,
            encoding = dataset.data_encoding,
            train_size = dataset.train_size,
            test_size = dataset.test_size 
        )

        return data_ingestion_config


In [26]:
import os
import zipfile
from sentimentAnalyzer.logging import logger
from sentimentAnalyzer.utils.common import read_dataset

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.source_dir, 'r') as zip_file:
            zip_file.extractall(unzip_path)
        
    def split_dataset(self):
        dataset_path = Path(self.config.dataset_file)
        encoding = self.config.encoding
        df = read_dataset(path=dataset_path, encoding=encoding)
        train_data, test_data = train_test_split(df,
                                                 train_size=self.config.train_size,
                                                 test_size=self.config.test_size,
                                                 random_state=42)

        train_data.to_csv(f"{self.config.train_data_dir}/train.csv", index = False)
        logger.info(f"Train Data has been stored at {self.config.train_data_dir}/train.csv")
        test_data.to_csv(f"{self.config.test_data_dir}/test.csv", index = False)
        logger.info(f"Test Data has been stored at {self.config.test_data_dir}/test.csv")
        
        


In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_zip_file()
    data_ingestion.split_dataset()
except Exception as e:
    print(e)

[2025-01-02 00:28:38,424: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 00:28:38,426: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 00:28:38,427: INFO: common: created directory at: artifacts]
[2025-01-02 00:28:38,430: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-02 00:28:38,430: INFO: common: created directory at: artifacts/data_ingestion/train]
[2025-01-02 00:28:38,430: INFO: common: created directory at: artifacts/data_ingestion/test]
[2025-01-02 00:28:40,353: INFO: common: Reading dataset from path: artifacts\data_ingestion\raw.csv]
[2025-01-02 00:28:45,695: INFO: common: Dataset loaded successfully with 1600000 rows and 6 columns.]
[2025-01-02 00:28:55,686: INFO: 2467245859: Train Data has been stored at artifacts/data_ingestion/train/train.csv]
[2025-01-02 00:28:58,041: INFO: 2467245859: Test Data has been stored at artifacts/data_ingestion/test/test.csv]


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [21]:
os.path.exists(data_ingestion_config.test_data_dir)

True

In [ ]:
data_ingestion_config.dataset_file

In [ ]:
df = pd.read_csv(data_ingestion_config.dataset_file, encoding="ISO-8859-1")
df.head()

In [16]:
from ensure import ensure_annotations

In [17]:
@ensure_annotations
def read_dataset(path: Path, encoding: str) -> pd.DataFrame:
    """
    Reads a CSV dataset into a pandas DataFrame.

    Args:
        path (Path): The file path to the dataset.
        encoding (str): The encoding format for the file. Defaults to 'utf-8'.

    Returns:
        pd.DataFrame: The loaded dataset.

    Raises:
        FileNotFoundError: If the file at the given path does not exist.
        ValueError: If an error occurs during file reading.
    """
    cols_name = ["target", "ids", "date", "flag", "user", "text"]
    if not os.path.exists(path):
        logger.error(f"File not found at path: {path}")
        raise FileNotFoundError(f"The file at {path} does not exist.")
    
    try:
        logger.info(f"Reading dataset from path: {path}")
        df = pd.read_csv(path, encoding=encoding, names=cols_name)
        logger.info(f"Dataset loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")
        return df
    except Exception as e:
        logger.error(f"Error occurred while reading the dataset: {e}")
        raise ValueError(f"Failed to read the dataset at {path}. Error: {e}")

In [ ]:
df_01 = read_dataset(Path(data_ingestion_config.dataset_file), encoding=data_ingestion_config.encoding)

In [ ]:
df_01.head()

In [ ]:
df_01.info()

In [21]:
X = df_01.iloc[:,1:]
y = df_01.iloc[:,0]

In [ ]:
y.head()

In [ ]:
X.head()

In [ ]:
data_ingestion_config.train_set

In [30]:
X_train, X_test = train_test_split(df_01,test_size=data_ingestion_config.test_size, random_state=40)

In [ ]:
X_train.head()

In [179]:
new_df = pd.concat([X_train, y_train], axis=1)

In [ ]:
new_df

In [183]:
new_df.to_csv("train.csv", index=False)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
type(X_train)